In [36]:
import sys
print(sys.executable)


/Users/gabrielmarec/anaconda3/envs/ml_env/bin/python


In [37]:
import mlflow
import pandas as pd
import os
from dvc.api import open as dvc_open
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle
import dagshub

In [38]:
dagshub.init(repo_owner='Nagisa', repo_name='ml_ops_project', mlflow=True)

Initialized MLflow to track repo "Nagisa/ml_ops_project"

Repository Nagisa/ml_ops_project initialized!

In [39]:
# Charger les données à partir des fichiers gérés avec DVC
train_path = "data/train.csv"
test_path = "data/test.csv"

# Vérifier si les fichiers existent localement via DVC
if not os.path.exists(train_path) or not os.path.exists(test_path):
    raise FileNotFoundError("Les fichiers train.csv ou test.csv sont introuvables. Assurez-vous qu'ils sont gérés par DVC et récupérés avec 'dvc pull'.")

# Charger les données avec DVC
with dvc_open(train_path, "r") as f:
    train_data = pd.read_csv(f)

with dvc_open(test_path, "r") as f:
    test_data = pd.read_csv(f)

CloneError: SCM error

In [9]:

# Prétraitement des données
train_data.fillna(train_data.median(), inplace=True)
test_data.fillna(test_data.median(), inplace=True)

# Définir les features et la cible
features = [
    "MSSubClass", "LotFrontage", "LotArea", "OverallQual", "OverallCond",
    "YearBuilt", "YearRemodAdd", "GrLivArea", "FullBath", "HalfBath", "BedroomAbvGr"
]
target = "SalePrice"

X = train_data[features]
y = train_data[target]

# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Démarrer une expérience MLflow
with mlflow.start_run():
    # Entraîner un modèle RandomForest pour la régression
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Faire des prédictions sur l'ensemble de validation
    y_pred = model.predict(X_val)

    # Calculer la métrique d'erreur (RMSE)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print(f"RMSE sur l'ensemble de validation : {rmse}")

    # Enregistrer les métriques et le modèle dans MLflow
    mlflow.log_param("model_type", "RandomForestRegressor")
    mlflow.log_metric("rmse", rmse)

    # Sauvegarder le modèle dans un fichier pickle
    model_filename = "random_forest_model.pkl"
    with open(model_filename, "wb") as f:
        pickle.dump(model, f)
    print(f"Modèle sauvegardé dans le fichier : {model_filename}")

# Utilisation du modèle pour des prédictions sur le test dataset
test_features = test_data[features]
test_predictions = model.predict(test_features)

# Préparer un fichier de soumission
submission = pd.DataFrame({
    "Id": test_data["Id"],
    "SalePrice": test_predictions
})
submission_path = "sample_submission.csv"
submission.to_csv(submission_path, index=False)
print(f"Fichier de soumission sauvegardé : {submission_path}")



/Users/gabrielmarec/anaconda3/envs/ml_env/bin/python


Initialized MLflow to track repo "Nagisa/ml_ops_project"

Repository Nagisa/ml_ops_project initialized!